In [1]:
import time
import sqlite3
import requests
import datetime
import pandas as pd
import shioaji as sj
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar
import mplfinance as mpf
import talib

import finlab
from finlab.online.sinopac_account import SinopacAccount
from finlab import data
from finlab.backtest import sim
finlab.login('qJ5mydSbDgtQLXFOf7QNTs7TOTexbhAbhgFIQ9CmXyifNbb45nxtM/RV7Kwi4T49#free')

from FuncBase import *

輸入成功!
輸入成功!


In [2]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(os.getenv('YOUR_PERSON_ID'),os.getenv('YOUR_PASSWORD'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
<SecurityType.Index: 'IND'> fetch done
<SecurityType.Stock: 'STK'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done


In [3]:
#設定callback 回傳資料型態
@api.on_tick_fop_v1()

def quote_callback(exchange: Exchange, tick: TickFOPv1):

    ticks = {}
    volume_today = {}

    ts = pd.to_datetime(tick.datetime)

    if tick.code[0].isalpha():
        code = tick.code[0:3]
    else:
        code = tick.code

    ts = pd.to_datetime(tick.datetime)

    if 5 < ts.hour < 9:
        return
    
    close = float(tick.close)
    volume = tick.volume
    vol_sum = tick.total_volume

    
    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

In [8]:
ticks = {}
kbars = {}
volume_today = {}

In [13]:
import time

#tbot: 加入即時訂閱，即時更新tick資料
def tbot(codes, is_Futures = False):

    print('--- 啟動 Tbot for {} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')

    date = pd.to_datetime(datetime.datetime.now().date())

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return

    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    else:
        code = codes

    prev_trading_dates = tw_calendar.sessions_window(date, -3)[0]
    #for pre_trading_days in prev_trading_dates:
    #    prev_ticks[code] = pd.concat([prev_ticks[code],get_ticks(connection, api, pre_trading_days, codes , is_Futures )[0]])

    
    ticks[code] = get_ticks(connection, api, date, codes , is_Futures )[0]
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
    kbars[code] = kbars[code][prev_trading_dates:]
    volume_today[code] = kbars[code]['volume'].sum()


    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0 :

            kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
            kbars[code] = kbars[code][prev_trading_dates:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

            try:
                current_kbars = kbars[code].loc[str_current_time]
            except:
                continue

            print(current_kbars)

In [ ]:
tbot('TMF202410', True)

In [5]:
api.quote.subscribe(api.Contracts.Futures.TMF['TMF202410'], quote_type='tick',
                    version = sj.constant.QuoteVersion.v1
)

In [6]:
api.quote.unsubscribe(
    api.Contracts.Futures.TMF['TMF202410'],
    quote_type = sj.constant.QuoteType.Tick,
    version = sj.constant.QuoteVersion.v1
)

In [ ]:
api.quote.unsubscribe(
    api.Contracts.Futures.TMF['TMF202410'],
    quote_type = sj.constant.QuoteType.Tick,
    version = sj.constant.QuoteVersion.v1
)

In [13]:
def get_MA(kbars):
    import talib  

    kbars['ma5'] = talib.MA(kbars.close, 5) 
    kbars['ma10'] = talib.MA(kbars.close, 10)
    kbars['ma20'] = talib.MA(kbars.close, 20)  

    return kbars


In [15]:
def trend_trading(connection, api, start_date, end_date,
                  codes = str, is_Futures = False,
                  filter = 0.1, Ratio_Drawdown = 0.1, Quit_position = 0
):
    tw_calendar = get_calendar('XTAI')

    #Risk Variables
    filter = 0.1
    cost_after_add = 0
    Ratio_Drawdown1 = -0.08 #預設台股最大漲跌幅
    Ratio_Drawdown2 = -0.03
    Quit_position = 0

    #Situation Variable
    entry_time = None
    entry_price = 0

    exit_time = None
    exit_price = 0

    position = 0
    add_position = 0
    last_price = 0
    #平均成本
    AC = 0
    #Entry High
    EH = 0

    transactions = pd.DataFrame(columns=[
        '成交日期',
        '股票代號',
        '進場時間',
        '平均買進價格',
        '總買進口數',
        '賣出時間',
        '賣出價格',
        '總賣出口數',
        '賣出價格',
        '價差'
    ])
    all_kbar = historical_kbars(connection, api, start_date, end_date,
                      '30Min', codes, is_Futures)
    all_kbar = get_MA(all_kbar)
    all_kbar.index = pd.to_datetime(all_kbar.index)

    for date in pd.date_range(start_date, end_date):
        
        if date not in tw_calendar.opens:
            continue
        
        date = pd.to_datetime(date)
        #ticks = get_ticks(connection, api, date, codes, is_Futures)[0]
        #kbars = ticks_to_kbars(ticks, '30Min')
        #kbars = get_MA(all_kbar)
        
        kbars = all_kbar[date:date+pd.Timedelta(days=1)]
        #print( 'Now: [{}]: {} for {} units at {} and the Entry high is {}' .format(date, codes, position, round(AC,2), EH))

        for ts in range(len(kbars)):
            
            #Entry Condition
            current_time = kbars.iloc[ts].name + pd.Timedelta(minutes=30)
            current_price = kbars['close'][ts]
            buy_price = kbars['open'][ts]

            if current_price >= EH:
                    EH = current_price

            if (
                #first condition
                position == 0 and 
                #OR condition
                #current_price/kbars.iloc[ts]['ma20']-1 < filter or 
                current_price > kbars.iloc[ts]['ma20']                        
            ):
                if (
                    #Entry condition
                    current_price > kbars.iloc[ts]['ma10']         
                ):
                    position = 1

                    entry_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    entry_price = current_price

                    #Cost record
                    AC = current_price
                    EH = current_price
                    last_price = current_price
                    
                    #Report
                    print( 'Entry: [{}] buy {} at {}'.format(current_time, codes, current_price))

                continue
            
            # 進行加碼或出場
            if(
                position != 0
            ):
                add_position = 1
                cost_after_add = (AC * position + current_price * add_position) / (position + add_position)
                cv1 = EH * (1+Ratio_Drawdown1) / cost_after_add -1
                cv2 = EH * (1 + Ratio_Drawdown1) 
                
                # 加碼條件1
                if cv1 > 0:

                    #Cost record
                    AC = (AC * position + current_price * add_position) / (position + add_position)
                    EH = current_price
                    position += add_position
                    last_price = current_price

                        
                    #Report
                    print( 'Entry: [{}] Reach condition1: {} and buy {} at {}'.format(current_time,  round(cv1, 5),
                                                                                    codes, current_price))
                    print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))
                    


                #加碼條件2
                elif cv2 > last_price:

                    #Cost record
                    AC = (AC * position + current_price * add_position) / (position + add_position)
                    EH = current_price
                    position += add_position
                    last_price = current_price

                        
                    #Report
                    print( 'Entry: [{}] Reach condition2: {} and buy {} at {}'.format(current_time,  round(cv2, 5),
                                                                                    codes, current_price))
                    print( 'Now: [{}] buy {} for {} units in at average price of {} '.format(current_time, codes, position, round(AC,2)))
                    

                #下下一根k open price
                #出場條件1
                elif(
                    position < 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):

                    exit_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    exit_price = current_price

                    print( 'Exit: [{}] Reach condition1: {} and sell {} for {} units at {}'.format(current_time,
                                                                                                   round(EH * (1+Ratio_Drawdown1), 5),
                                                                                                  codes, position, current_price))
                    
                
                #出場條件2
                elif(
                    position > 5 and
                    current_price <= EH * (1+Ratio_Drawdown2)
                ):
                    exit_time = pd.Timestamp(date) + pd.Timedelta(hours=current_time.hour, minutes=current_time.minute, seconds=current_time.second)
                    exit_price = current_price


                    print( 'Exit: [{}] Reach condition2: {} and sell {} for {} units at {}'.format(current_time, 
                                                                                                  round(EH * (1+Ratio_Drawdown2),5),
                                                                                                  codes, position, current_price))

                
            
            if entry_time and exit_time:
                transaction = pd.DataFrame([[date,
                                            codes,
                                            entry_time,
                                            AC,
                                            position ,
                                            exit_time,
                                            exit_price,
                                            position,
                                            exit_price,
                                            (exit_price-entry_price)*position*10]                                 
                                            ],
                                            columns = [
                                                '成交日期',
                                                '股票代號',
                                                '進場時間',
                                                '平均買進價格',
                                                '總買進口數',
                                                '賣出時間',
                                                '賣出價格',
                                                '總賣出口數',
                                                '賣出價格',
                                                '價差'])
                # Append this transaction to the overall transactions dataframe
                transactions = pd.concat([transactions, transaction], ignore_index=True)

                # Reset for the next trade
                AC = 0
                EH = 0
                position = 0
                entry_time = None
                exit_time = None

    # Return all transactions at the end
    return transactions


1.可以跑策略但還不能有多次交易<br>
2.加碼點位問題(30Min k 隔太久，價位差很多)<br>

In [17]:
codes = 'TMF202410'

trend_trading(connection, api, '2024-6-1', '2024-10-1', codes, True)

Entry: [2024-07-30 12:00:00] buy TMF202410 at 22132.0
Exit: [2024-08-02 09:00:00] Reach condition1: 20930.0 and sell TMF202410 for 1 units at 21972.0
Entry: [2024-08-05 22:00:00] buy TMF202410 at 19510.0
Entry: [2024-08-07 10:30:00] Reach condition2: 19554.6 and buy TMF202410 at 21255.0
Now: [2024-08-07 10:30:00] buy TMF202410 for 2 units in at average price of 20382.5 
Exit: [2024-08-08 04:00:00] Reach condition1: 19686.16 and sell TMF202410 for 2 units at 20756.0
Entry: [2024-08-08 11:30:00] buy TMF202410 at 21060.0
Exit: [2024-09-03 22:00:00] Reach condition1: 20732.2 and sell TMF202410 for 1 units at 21828.0
Entry: [2024-09-04 20:30:00] buy TMF202410 at 21135.0
Exit: [2024-09-06 23:30:00] Reach condition1: 19717.44 and sell TMF202410 for 1 units at 20786.0
Entry: [2024-09-09 09:30:00] buy TMF202410 at 20888.0
Entry: [2024-09-24 21:00:00] Reach condition2: 20889.52 and buy TMF202410 at 22706.0
Now: [2024-09-24 21:00:00] buy TMF202410 for 2 units in at average price of 21797.0 
Exit:

,成交日期,股票代號,進場時間,平均買進價格,總買進口數,賣出時間,賣出價格,總賣出口數,賣出價格,價差
0,2024-08-02,TMF202410,2024-07-30 12:00:00,22132.0,1,2024-08-02 09:00:00,21972.0,1,21972.0,-1600.0
1,2024-08-08,TMF202410,2024-08-05 22:00:00,20382.5,2,2024-08-08 04:00:00,20756.0,2,20756.0,24920.0
2,2024-09-03,TMF202410,2024-08-08 11:30:00,21060.0,1,2024-09-03 22:00:00,21828.0,1,21828.0,7680.0
3,2024-09-06,TMF202410,2024-09-04 20:30:00,21135.0,1,2024-09-06 23:30:00,20786.0,1,20786.0,-3490.0
4,2024-09-30,TMF202410,2024-09-09 09:30:00,21797.0,2,2024-09-30 10:00:00,22632.0,2,22632.0,34880.0
